<a href="https://colab.research.google.com/github/sindla97/Machine-Learning/blob/main/NER_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
#importing library
import pandas as pd
import numpy as np
import tensorflow as tf

In [1]:
#!pip install opendatasets

import opendatasets as od

od.download(
	"https://www.kaggle.com/datasets/naseralqaydeh/named-entity-recognition-ner-corpus")

Please provide your Kaggle credentials to download this dataset. Learn more: http://bit.ly/kaggle-creds
Your Kaggle username: hemanthshanmuka
Your Kaggle Key: ··········


100%|██████████| 4.14M/4.14M [00:00<00:00, 4.73MB/s]


In [6]:
# reading the XLSX file
file =('/content/named-entity-recognition-ner-corpus/ner.csv')
ner = pd.read_csv(file)

In [10]:
ner.head()

,Sentence #,Sentence,POS,Tag
0,Sentence: 1,Thousands of demonstrators have marched throug...,"['NNS', 'IN', 'NNS', 'VBP', 'VBN', 'IN', 'NNP'...","['O', 'O', 'O', 'O', 'O', 'O', 'B-geo', 'O', '..."
1,Sentence: 2,Families of soldiers killed in the conflict jo...,"['NNS', 'IN', 'NNS', 'VBN', 'IN', 'DT', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
2,Sentence: 3,They marched from the Houses of Parliament to ...,"['PRP', 'VBD', 'IN', 'DT', 'NNS', 'IN', 'NN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
3,Sentence: 4,"Police put the number of marchers at 10,000 wh...","['NNS', 'VBD', 'DT', 'NN', 'IN', 'NNS', 'IN', ...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."
4,Sentence: 5,The protest comes on the eve of the annual con...,"['DT', 'NN', 'VBZ', 'IN', 'DT', 'NN', 'IN', 'D...","['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', ..."


In [16]:
len(ner['Sentence'][0].split()),len(ner['POS'][0]),len(ner['Tag'][0])

(24, 150, 132)